In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cufflinks as cf
import seaborn as sns
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.model_selection import learning_curve
from sklearn.model_selection import ShuffleSplit
%matplotlib inline

In [ ]:
train = pd.read_csv('final-bea-user1-tcc.csv')

In [ ]:
def change_chips(column):
    Chips = column[0]
    
    if Chips == 'Single Chip':
        return 1;
    elif Chips == 'Dual Chip':
        return 2;
    elif Chips == 'Tri Chip':
        return 3;
    elif Chips == 'Quadri Chip':
        return 4
    else:
        return Chips

In [ ]:
train['chips'] = train[['chips']].apply(change_chips, axis=1)

In [ ]:
def change_null_chips(column):
    Chips = column[0]
    
    if pd.isnull(Chips):
        return 1;
    else:
        return Chips

In [ ]:
train['chips'] = train[['chips']].apply(change_null_chips, axis=1)

In [ ]:
def change_null_rat(column):
    rating = column[0]
    
    if pd.isnull(rating):
        return 0;
    else:
        return rating

In [ ]:
train['max_price'] = train[['max_price']].apply(change_null_rat, axis=1)

In [ ]:
train['rating_number'] = train[['rating_number']].apply(change_null_rat, axis=1)

In [ ]:
train['rating'] = train[['rating']].apply(change_null_rat, axis=1)

In [ ]:
train['min_price'] = train[['min_price']].apply(change_null_rat, axis=1)

In [ ]:
sns.heatmap(train.isnull(), yticklabels=False,cbar=False, cmap='viridis')

In [ ]:
typeDevice = pd.get_dummies(train['type'], drop_first=True)

In [ ]:
train = pd.concat([train,typeDevice],axis=1)

In [ ]:
brand = pd.get_dummies(train['brand'])

In [ ]:
train = pd.concat([train,brand],axis=1)

In [ ]:
train.drop(['back_camera', 'operational_system', 'screen_size', 'image', 'name', 'brand', 'type', 'id', 'id_api'],axis=1,inplace=True)

In [ ]:
X = train.drop('liked', axis=1)
y = train['liked']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=101)
logmodel = LogisticRegression()
logmodel.fit(X_train, y_train)

In [ ]:
predictions = logmodel.predict(X_test)

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
print(confusion_matrix(y_test,predictions))

In [ ]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=1, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=None, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

title = "Learning Curves (Logistic Regression)"
# Cross validation with 100 iterations to get smoother mean test and train
# score curves, each time with 20% data randomly selected as a validation set.
cv = ShuffleSplit(n_splits=100, test_size=0.2, random_state=0)
plot_learning_curve(logmodel, title, X_train, y_train, cv=None)
plt.show()